### Setup

In [1]:
import pandas as pd
import numpy as np
import os
from time import time

In [2]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.linear_model import ElasticNetCV, LinearRegression, Lars,LassoLarsCV
from sklearn.svm import LinearSVR, SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, train_test_split, RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt


In [3]:
from sklearn.model_selection import cross_validate, train_test_split, RepeatedKFold, GridSearchCV

In [5]:
#import sys
#sys.path.append(os.path.abspath('..'))#sys.path[0] + '/..') 
from vb_estimators import LinRegSupreme,LinSVR,RBFSVR,ENet,L1Lars,GBR,HGBR,FlexiblePipe
from vb_helper import VBHelper
from vb_cross_validator import regressor_q_stratified_cv

SyntaxError: invalid syntax (vb_estimators.py, line 59)

In [ ]:

from warnings import filterwarnings
filterwarnings('ignore')


In [ ]:
gridpoints=5
kwargs=dict(
    test_share=0.2,
    cv_folds=10,
    cv_reps=10,
    cv_groupcount=10,
    cv_strategy='quantile', #'none',# for stratified cv
    random_state=2 # random_state for reproducibility
)
vbhelper=VBHelper(**kwargs)

In [ ]:
scorer_list=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'] #cross_validate wants strings
vbhelper.scorer_list=scorer_list

##### Example Dataset

In [ ]:
data_path=os.path.join('sample_data','ex1.csv')
df=pd.read_csv(data_path)
all_vars=list(df.columns)
print(all_vars)

In [ ]:
y_name='THG_Fish'
loc_vars=['LAT','LONG']
drop_vars=['Date','OrigHabCode','STA_ID']
drop_vars.extend(loc_vars)
drop_vars.append(y_name)
x_vars=[var for var in all_vars if var not in drop_vars]
X_df=df.loc[:,x_vars]
y_df=df.loc[:,y_name]

In [ ]:

shuf=np.arange(y_df.shape[0])
np.random.seed(0)
np.random.shuffle(shuf)
X_df=X_df.iloc[shuf]
y_df=y_df.iloc[shuf]
vbhelper.setData(X_df,y_df)
X_train, X_test, y_train, y_test=vbhelper.train_test_split()

In [ ]:
est_kwargs=dict(gridpoints=gridpoints,cat_idx=vbhelper.cat_idx,float_idx=vbhelper.float_idx,bestT=True)
estimator_dict={
    #'histogram-gradient-boosting-reg':HGBR,
    'expXB-least-sq':FlexiblePipe,
    'gradient-boosting-reg':GBR,
    #'lin-reg-supreme':lambda: LinRegSupreme(**est_kwargs),
    #'elastic-net':lambda: ENet(**est_kwargs), 
    #'linear-svr-cv':lambda: LinSVR(**est_kwargs), 
    'rbf-svr-cv':lambda: RBFSVR(**est_kwargs), 
    'lassolars':lambda: L1Lars(**est_kwargs),
    
    }
vbhelper.estimator_dict=estimator_dict
vbhelper.model_dict={key:val() for key,val in estimator_dict.items()} # they will be models once .fit is called

In [ ]:
i=0
for name,est in vbhelper.model_dict.items():
    start=time()
    i+=1;print(name)
    est.fit(X_train,y_train)
    print('train R2:',est.score(X_train,y_train))
    if not X_test is None:print('test R2:',est.score(X_test,y_test))
    end=time()
    print(f'runtime:{(end-start)/60} min.\n')


In [ ]:
vbhelper.runCrossValidate(n_jobs=4)

In [ ]:
#plot cv_yhat

#### graphs and table to summarize results

In [ ]:
vbhelper.buildCVScoreDict()

In [ ]:
vbhelper.viewCVScoreDict()

In [ ]:
vbhelper.plotCVScores(sort=1)

In [ ]:
vbhelper.predictCVYhat()

In [ ]:
vbhelper.plotCVYhat(single_plot=True)

In [ ]:
vbhelper.plotCVYhat(single_plot=False)